In [10]:
import os

repo_dir = "Movie-Recommender-System"

if os.path.exists(repo_dir):
    print(f"{repo_dir} already exists. Removing it...\n")
    !rm -r {repo_dir}

# Clone the repository from GitHub
!git clone https://github.com/Goshmar/Movie-Recommender-System

Movie-Recommender-System already exists. Removing it...

Cloning into 'Movie-Recommender-System'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 26 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (26/26), 6.67 MiB | 13.72 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [11]:
import requests
import zipfile
import pandas as pd
import numpy as np

# Define the paths
zip_file_path = "/content/Movie-Recommender-System/data/raw/ml-100k.zip"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(".")

# ZIP cleaning up
os.remove(zip_file_path)

In [12]:
from scipy.sparse import load_npz

# Download data
data_matrix = load_npz('/content/Movie-Recommender-System/data/interim/data_matrix.npz')
item_info = pd.read_csv('/content/ml-100k/u.item', sep='|', encoding='latin-1', names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDB_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

In [13]:
from scipy.sparse.linalg import svds
U, S, Vt = svds(data_matrix, k=50, return_singular_vectors='vh')

In [20]:
# Let's take sub-query for movie title for finding input index
query = 'Lord'

matching_rows = item_info.loc[item_info.movie_title.str.contains(query, flags=2)]

# Check if there are any matching rows
matching_index = matching_rows.index[0] if not matching_rows.empty else None

print("First matching index:", matching_index)
matching_rows

First matching index: 550


,movie_id,movie_title,release_date,video_release_date,IMDB_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
550,551,Lord of Illusions (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Lord%20of%20I...,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
966,967,Little Lord Fauntleroy (1936),01-Jan-1936,NaN,http://us.imdb.com/M/title-exact?Little%20Lord...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
item_info.iloc[np.argsort(-Vt.T @ Vt[:, matching_index])[1:6]][['movie_id', 'movie_title']]

,movie_id,movie_title
226,227,Star Trek VI: The Undiscovered Country (1991)
403,404,Pinocchio (1940)
161,162,On Golden Pond (1981)
385,386,Addams Family Values (1993)
568,569,Wolf (1994)
